In [1]:
import pandas as pd
import os
import requests, sys
import re
import csv


## Use QuickGO API to download 

In [ ]:
def getGoTerms(URL):
    requestURL = "https://www.ebi.ac.uk/QuickGO/services/annotation/downloadSearch?geneProductId=" + URL
    r = requests.get(requestURL, headers={ "Accept" : "text/tsv"})
    if not r.ok:
        responseBody = "!!!!!!!!!!!!Too many parameters\n"
        return (responseBody)
        #r.raise_for_status()
        #sys.exit()
   
    responseBody = r.text
    return (responseBody)


### The code below opens the ECDataSet file and reads it line by line, each line has ECNumber: Space separated enzymes 
### Splits each line at colon with ECNum[0]  = ECNumber and ECNum[1] = list of proteins
### Replace Space between proteins with %2C in each ECNUM[1] as per the API requirements for URL.
### Append EC number, newline and the resulting response from API to ECGotermsFile.

filename = 'ECdataset.txt'
ECGotermsFile = 'AllECdatasetGoterms.txt'
with open(filename) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 
for ECNum in content:
    ECNum = re.split('[:]', ECNum)
    print(ECNum[0])
    ECNum[1] = ECNum[1][1:].replace(" ", "%2C")
    file1 = open(ECGotermsFile,"a")    
    file1.write(ECNum[0] + '\n')
    file1.write(getGoTerms(ECNum[1]))
    file1.close()


## Filter GO Terms According to Aspect and Evidence Code

In [2]:
def ReadGOTermsFile (r_filenameTSV, aspect, IEA):
 
    # read the data
    tsv_read = pd.read_csv(r_filenameTSV, sep='\t')
    #print(tsv_read.head(10))
    #drop unneccessary columns
    tsv_read = tsv_read.drop(['GENE PRODUCT DB','ECO ID', 'REFERENCE', 'WITH/FROM', 'TAXON ID', 'ASSIGNED BY', 'ANNOTATION EXTENSION' ], axis=1)
    
    if IEA == 'Non-IEA':
        # Filter out the IEA annotations and keep only one Aspect(Ontology MF, CC, BP)
        if aspect== 'MF':
            Filtered = tsv_read [(tsv_read['GO EVIDENCE CODE'] != 'IEA') & (tsv_read['GO ASPECT'] == 'F')]
        if aspect=='BP':
            Filtered = tsv_read [(tsv_read['GO EVIDENCE CODE'] != 'IEA') & (tsv_read['GO ASPECT'] == 'P')]
        if aspect== 'CC':
            Filtered = tsv_read [(tsv_read['GO EVIDENCE CODE'] != 'IEA') & (tsv_read['GO ASPECT'] == 'C')]
        # Drop rows with duplicate GO terms
        return Filtered.drop_duplicates('GO TERM')


    elif IEA == 'IEA':
        if aspect== 'MF':
            Filtered = tsv_read [ (tsv_read['GO ASPECT'] == 'F')]
        if aspect=='BP':
            Filtered = tsv_read [(tsv_read['GO ASPECT'] == 'P')]
        if aspect== 'CC':
            Filtered = tsv_read [(tsv_read['GO ASPECT'] == 'C')]
        # Drop rows with duplicate GO terms
        return Filtered.drop_duplicates('GO TERM')
# Open function to open the file "MyFile1.txt"  
# (same directory) in append mode and 
def GO_WriteToFile (GOTermsFile, tsv_read, aspect,filename ):
    file1 = open(GOTermsFile,"a") 
    #L = [tsv_read["GENE PRODUCT ID"][1]," ", aspect, ":" ]
    #file1.writelined(L)
    #print(tsv_read["GENE PRODUCT ID"])
    x = tsv_read['GO TERM'].tolist()
    print (x)
    #file1.writelines(tsv_read['GO TERM'].tolist()) 
    file1.write(filename + '\n')
    for y in x:
        file1.write(" '"+y+"',")
    file1.write("\n")
    file1.close() 

In [3]:
def getGOTerms (ECdataframe, aspect, IEA):
    
    if IEA == 'Non-IEA':
        # Filter out the IEA annotations and keep only one Aspect(Ontology MF, CC, BP)
        if aspect== 'MF':
            Filtered = ECdataframe [(ECdataframe['GO EVIDENCE CODE'] != 'IEA') & (ECdataframe['GO ASPECT'] == 'F')]
            #print (Filtered)
        if aspect=='BP':
            Filtered = ECdataframe [(ECdataframe['GO EVIDENCE CODE'] != 'IEA') & (ECdataframe['GO ASPECT'] == 'P')]
        if aspect== 'CC':
            Filtered = ECdataframe [(ECdataframe['GO EVIDENCE CODE'] != 'IEA') & (ECdataframe['GO ASPECT'] == 'C')]
       
    elif IEA == 'IEA':
        if aspect == 'MF':
            Filtered = ECdataframe [ (ECdataframe['GO ASPECT'] == 'F')]
        if aspect=='BP':
            Filtered = ECdataframe [(ECdataframe['GO ASPECT'] == 'P')]
        if aspect== 'CC':
            Filtered = ECdataframe [(ECdataframe['GO ASPECT'] == 'C')]
    return Filtered

def GO_WriteToFile (GOTermsFile, ECdataframe, ECNum, aspect):
    file1 = open(GOTermsFile,"a")    
    file1.write(ECNum + '\n')
    x = ECdataframe.groupby('GENE PRODUCT ID')['GO TERM'].agg(lambda x: ','.join(x.unique())).reset_index()
    file1.write(x.to_csv(sep='\t',index=False, header=False))    
    file1.close()

AllECData = {}
regex = r"[0-7][.][0-9]+[.][0-9]+[.][0-9]+"
#Read_filename = 'ALL_EC_GOTERMS.tsv'
Read_filename = 'AllECdatasetGoterms.txt'
#WriteFile = 'ALL_EC_GOTERMS_NONIEA_MF.tsv'
WriteFile = 'ALL_EC_GOTERMS_IEA_MF.tsv'
#WriteFile = 'ALL_EC_GOTERMS_IEA_BP.tsv'
#WriteFile = 'ALL_EC_GOTERMS_NONIEA_BP.tsv'

#WriteFile = 'ALL_EC_GOTERMS_Edited.txt'
with open( Read_filename, "r" ) as source:
    tsvreader = csv.reader(source, delimiter="\t")
    for line in tsvreader:
        # use re.match() instead of re.search() becuase match only searches for pattern match at the beginning of the string.
        #print(line[0])
        if(re.match(regex, line[0])):
            QuickGOAnnotations = []
            AllECData[line[0]] = QuickGOAnnotations
        else:    
            QuickGOAnnotations.append(line)

aspectM = 'MF'
aspectB = 'BP'
aspectC = 'CC'
IEA_N = 'Non-IEA'
IEA = 'IEA'
for ECNum, Annotations in AllECData.items():
    print(ECNum)
    headers = AllECData[ECNum][0]
    ECdataframe = pd.DataFrame(AllECData[ECNum][1:], columns = headers)
    #ECdataframe = pd.DataFrame(AllECGoterms[ECNum])
    ECdataframe = ECdataframe.drop(['GENE PRODUCT DB','ECO ID', 'REFERENCE', 'WITH/FROM', 'TAXON ID', 'ASSIGNED BY', 'ANNOTATION EXTENSION' ], axis='columns')
    Filtered = getGOTerms(ECdataframe, aspectM, IEA)

    #GO_WriteToFile("ALL_EC_GOTERMS_NonIEA.tsv", Filtered,ECNum, aspectM)
    GO_WriteToFile(WriteFile, Filtered, ECNum, aspectM)

6.1.1.9
3.2.1.1
3.2.1.26
3.1.3.1
2.7.1.48
1.14.18.1
4.2.99.18
3.4.16.6
6.1.1.11
6.3.4.21
3.6.1.66
1.18.6.1
1.8.1.9
2.5.1.18
4.1.3.27
1.6.5.2
1.12.98.3
6.1.1.5
3.6.4.13
3.2.1.23
2.7.10.1
3.6.4.12
2.7.7.48
2.7.13.3
3.6.1.23
4.1.1.31
2.7.7.7
6.1.1.7
3.4.21.53
2.3.2.27
6.1.1.4
2.7.7.6
2.7.1.11
6.5.1.1
3.1.3.48
2.7.11.13
2.7.7.49
3.4.19.12
1.5.3.1
4.6.1.2
3.2.1.4
3.2.1.20
2.7.1.23
3.1.4.53
2.7.12.1
2.1.1.43
2.7.11.21
3.6.5.5
2.7.1.67
2.3.2.26
3.1.3.16
2.7.1.1
5.6.2.2
3.1.13.1
4.1.1.65
3.1.1.11
1.17.4.1
2.7.11.25
7.1.2.2
6.1.1.20
3.2.1.14
3.5.1.98
3.2.1.8
2.7.11.22
3.6.1.3
5.6.2.3
2.7.7.59
1.14.11.27
3.2.1.21
2.4.1.109
2.7.1.107
3.1.26.11
3.1.4.17
2.4.1.41
2.7.10.2
6.5.1.2
3.2.1.22
3.1.4.4
3.1.3.4
4.3.2.1
3.4.11.9
6.1.1.10
2.7.11.24
2.1.1.37
2.1.1.113
3.1.1.3
2.3.2.23
3.1.1.5
2.7.1.33
3.1.4.11
3.2.1.17
3.1.3.12
2.3.1.15
3.2.1.52
6.1.1.21
3.1.22.4
6.1.1.17
4.2.1.33
3.5.2.7
3.2.1.39
2.1.3.15
3.1.1.4
3.1.26.5
1.9.3.1
3.1.13.4
3.1.4.12
2.3.1.48
4.6.1.1
3.5.1.2
6.3.2.2
3.1.4.52
2.3.1.193
2.7.7.19

## Convert Annotations to Named Tuples required for code

In [50]:
regex = r"[0-7][.][0-9]+[.][0-9]+[.][0-9]+"

regex1 = r"^[('A-Z 0-9_-]{4,}"

regex2 = r"GO:[0-9]{7}"
subst = "#\\g<0>"
subst1 = "\\g<0> = Gene('\\g<0>', ["
subst2 = "'\\g<0>'"
Read_filename = 'ALL_EC_GOTERMS_IEA_MF.tsv'
WriteFile = 'ALL_EC_GOTERMS_IEA_MF_edited.py'
AllECData = {}


with open( Read_filename, "r" ) as source:
    tsvreader = csv.reader(source, delimiter="\t")
    file1 = open(WriteFile,"a")    
    for line in tsvreader:
        # use re.match() instead of re.search() becuase match only searches for pattern match at the beginning of the string.
        #print(line[0])
        if(re.match(regex, line[0])):
            line[0] = re.sub(regex, subst, line[0], 0, re.MULTILINE)
            QuickGOAnnotations = []
            AllECData[line[0]] = QuickGOAnnotations
            file1.write(line[0] + '\n')    

        else:
            if(re.match(regex1, line[0])):
                line[0] = re.sub(regex1, subst1, line[0], 0, re.MULTILINE)
            #print(line[1])
             #   QuickGOAnnotations.append(line)
            if((regex2, line[1])):
                line[1] = re.sub(regex2, subst2, line[1], 0, re.MULTILINE)
            QuickGOAnnotations.append(line)
            
            for x in line:
                file1.write(x)    
            file1.write(']' + '\n')   
    file1.close()
